In [ ]:
# Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import lightgbm as lgb
import xgboost as xgb
import optuna
import shap
import warnings
warnings.filterwarnings('ignore')

# Plotly
import plotly.express as px
import plotly.graph_objects as go

# Style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("✓ Libraries loaded")

## 1. Data Loading & Exploration

In [ ]:
# Load data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"\nFirst 5 rows:")
train_df.head()

In [ ]:
# Column groups
date_col = 'date_id'
target_col = 'market_forward_excess_returns'

# Determine feature groups
feature_cols = [col for col in train_df.columns if col not in [date_col, target_col]]

# Group by prefixes
feature_groups = {}
for col in feature_cols:
    prefix = col.split('_')[0] if '_' in col else 'other'
    if prefix not in feature_groups:
        feature_groups[prefix] = []
    feature_groups[prefix].append(col)

print("Feature Groups:")
for prefix, cols in sorted(feature_groups.items()):
    print(f"  {prefix}: {len(cols)} features")

print(f"\nTotal feature count: {len(feature_cols)}")

In [ ]:
# Basic statistics
print("Target Variable Statistics:")
print(train_df[target_col].describe())

# Target distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].hist(train_df[target_col], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Target Distribution')
axes[0].set_xlabel('market_forward_excess_returns')
axes[0].set_ylabel('Frequency')

axes[1].boxplot(train_df[target_col])
axes[1].set_title('Target Boxplot')
axes[1].set_ylabel('market_forward_excess_returns')

plt.tight_layout()
plt.show()

In [ ]:
# Missing values check
missing_counts = train_df.isnull().sum()
missing_pct = (missing_counts / len(train_df)) * 100

missing_df = pd.DataFrame({
    'Missing_Count': missing_counts[missing_counts > 0],
    'Missing_Pct': missing_pct[missing_counts > 0]
}).sort_values('Missing_Pct', ascending=False)

if len(missing_df) > 0:
    print(f"{len(missing_df)} columns contain missing values:")
    print(missing_df.head(10))
else:
    print("✓ No missing values")

## 2. Feature Engineering — Technical Indicators

In [ ]:
def add_technical_indicators(df, feature_cols, windows=[5, 10, 20]):
    """
    Add technical indicators:
    - RSI (Relative Strength Index)
    - MACD (Moving Average Convergence Divergence)
    - EMA (Exponential Moving Average)
    - Bollinger Bands
    """
    df_copy = df.copy()
    
    # Take group-wise mean of features
    for prefix in ['D', 'E', 'I', 'M', 'P', 'S', 'V']:
        group_cols = [col for col in feature_cols if col.startswith(prefix + '_')]
        if len(group_cols) > 0:
            group_mean = df_copy[group_cols].mean(axis=1)
            
            for window in windows:
                # Rolling Mean & Std
                df_copy[f'{prefix}_rolling_mean_{window}'] = group_mean.rolling(window=window, min_periods=1).mean()
                df_copy[f'{prefix}_rolling_std_{window}'] = group_mean.rolling(window=window, min_periods=1).std()
                
                # EMA
                df_copy[f'{prefix}_ema_{window}'] = group_mean.ewm(span=window, adjust=False).mean()
            
            # RSI (14 period)
            delta = group_mean.diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=14, min_periods=1).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=14, min_periods=1).mean()
            rs = gain / (loss + 1e-10)
            df_copy[f'{prefix}_rsi_14'] = 100 - (100 / (1 + rs))
            
            # MACD
            ema12 = group_mean.ewm(span=12, adjust=False).mean()
            ema26 = group_mean.ewm(span=26, adjust=False).mean()
            df_copy[f'{prefix}_macd'] = ema12 - ema26
            df_copy[f'{prefix}_macd_signal'] = df_copy[f'{prefix}_macd'].ewm(span=9, adjust=False).mean()
            
            # Bollinger Bands
            rolling_mean_20 = group_mean.rolling(window=20, min_periods=1).mean()
            rolling_std_20 = group_mean.rolling(window=20, min_periods=1).std()
            df_copy[f'{prefix}_bb_upper'] = rolling_mean_20 + (2 * rolling_std_20)
            df_copy[f'{prefix}_bb_lower'] = rolling_mean_20 - (2 * rolling_std_20)
            df_copy[f'{prefix}_bb_width'] = df_copy[f'{prefix}_bb_upper'] - df_copy[f'{prefix}_bb_lower']
    
    return df_copy

print("Feature engineering function ready")

In [ ]:
# Add technical indicators
train_enhanced = add_technical_indicators(train_df, feature_cols)
test_enhanced = add_technical_indicators(test_df, feature_cols)

print(f"Original train shape: {train_df.shape}")
print(f"Enhanced train shape: {train_enhanced.shape}")
print(f"Number of added features: {train_enhanced.shape[1] - train_df.shape[1]}")

## 3. Data Preparation & Split

In [ ]:
# Split features and target
X_train = train_enhanced.drop(columns=[date_col, target_col])
y_train = train_enhanced[target_col]
X_test = test_enhanced.drop(columns=[date_col])

# Fill missing values
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_train.mean())  # use train mean for test

# Clean infinite values
X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(X_train.mean())
X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(X_train.mean())

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")

In [ ]:
# Create validation sets with TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train), 1):
    print(f"Fold {fold}: Train={len(train_idx)}, Val={len(val_idx)}")

## 4. Model Training & Evaluation

In [ ]:
def calculate_metrics(y_true, y_pred):
    """Compute model performance metrics"""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # Direction Accuracy
    direction_true = np.sign(y_true)
    direction_pred = np.sign(y_pred)
    direction_acc = np.mean(direction_true == direction_pred)
    
    return {
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'Direction_Accuracy': direction_acc
    }

print("Metric function ready")

In [ ]:
# Scaler (leakage-safe)
scaler = RobustScaler()

# Use last fold for validation
train_idx, val_idx = list(tscv.split(X_train))[-1]

X_tr = X_train.iloc[train_idx]
y_tr = y_train.iloc[train_idx]
X_val = X_train.iloc[val_idx]
y_val = y_train.iloc[val_idx]

# Scale
X_tr_scaled = scaler.fit_transform(X_tr)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"Train: {X_tr_scaled.shape}, Val: {X_val_scaled.shape}")

### 4.1 Ridge Regression

In [ ]:
# Ridge model
ridge_model = Ridge(alpha=1.0, random_state=42)
ridge_model.fit(X_tr_scaled, y_tr)

# Predictions
ridge_train_pred = ridge_model.predict(X_tr_scaled)
ridge_val_pred = ridge_model.predict(X_val_scaled)

# Metrics
ridge_train_metrics = calculate_metrics(y_tr, ridge_train_pred)
ridge_val_metrics = calculate_metrics(y_val, ridge_val_pred)

print("Ridge - Train Metrics:")
for k, v in ridge_train_metrics.items():
    print(f"  {k}: {v:.4f}")

print("\nRidge - Val Metrics:")
for k, v in ridge_val_metrics.items():
    print(f"  {k}: {v:.4f}")

### 4.2 LightGBM

In [ ]:
# LightGBM model
lgb_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'random_state': 42
}

lgb_train = lgb.Dataset(X_tr_scaled, y_tr)
lgb_val = lgb.Dataset(X_val_scaled, y_val, reference=lgb_train)

lgb_model = lgb.train(
    lgb_params,
    lgb_train,
    num_boost_round=500,
    valid_sets=[lgb_train, lgb_val],
    valid_names=['train', 'val'],
    callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(50)]
)

# Predictions
lgb_train_pred = lgb_model.predict(X_tr_scaled, num_iteration=lgb_model.best_iteration)
lgb_val_pred = lgb_model.predict(X_val_scaled, num_iteration=lgb_model.best_iteration)

# Metrics
lgb_train_metrics = calculate_metrics(y_tr, lgb_train_pred)
lgb_val_metrics = calculate_metrics(y_val, lgb_val_pred)

print("\nLightGBM - Train Metrics:")
for k, v in lgb_train_metrics.items():
    print(f"  {k}: {v:.4f}")

print("\nLightGBM - Val Metrics:")
for k, v in lgb_val_metrics.items():
    print(f"  {k}: {v:.4f}")

### 4.3 XGBoost

In [ ]:
# XGBoost model
xgb_params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 6,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42,
    'verbosity': 0
}

xgb_train = xgb.DMatrix(X_tr_scaled, label=y_tr)
xgb_val = xgb.DMatrix(X_val_scaled, label=y_val)

evals = [(xgb_train, 'train'), (xgb_val, 'val')]
xgb_model = xgb.train(
    xgb_params,
    xgb_train,
    num_boost_round=500,
    evals=evals,
    early_stopping_rounds=50,
    verbose_eval=50
)

# Predictions
xgb_train_pred = xgb_model.predict(xgb_train, iteration_range=(0, xgb_model.best_iteration))
xgb_val_pred = xgb_model.predict(xgb_val, iteration_range=(0, xgb_model.best_iteration))

# Metrics
xgb_train_metrics = calculate_metrics(y_tr, xgb_train_pred)
xgb_val_metrics = calculate_metrics(y_val, xgb_val_pred)

print("\nXGBoost - Train Metrics:")
for k, v in xgb_train_metrics.items():
    print(f"  {k}: {v:.4f}")

print("\nXGBoost - Val Metrics:")
for k, v in xgb_val_metrics.items():
    print(f"  {k}: {v:.4f}")

## 5. Model Comparison

In [ ]:
# Aggregate results
results_df = pd.DataFrame({
    'Model': ['Ridge', 'LightGBM', 'XGBoost'],
    'Train_RMSE': [ridge_train_metrics['RMSE'], lgb_train_metrics['RMSE'], xgb_train_metrics['RMSE']],
    'Val_RMSE': [ridge_val_metrics['RMSE'], lgb_val_metrics['RMSE'], xgb_val_metrics['RMSE']],
    'Train_DirAcc': [ridge_train_metrics['Direction_Accuracy'], lgb_train_metrics['Direction_Accuracy'], xgb_train_metrics['Direction_Accuracy']],
    'Val_DirAcc': [ridge_val_metrics['Direction_Accuracy'], lgb_val_metrics['Direction_Accuracy'], xgb_val_metrics['Direction_Accuracy']],
})

print("\n" + "="*70)
print("MODEL COMPARISON")
print("="*70)
print(results_df.to_string(index=False))
print("="*70)

In [ ]:
# Görselleştirme
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# RMSE karşılaştırması
x = np.arange(len(results_df))
width = 0.35

axes[0].bar(x - width/2, results_df['Train_RMSE'], width, label='Train', alpha=0.8)
axes[0].bar(x + width/2, results_df['Val_RMSE'], width, label='Val', alpha=0.8)
axes[0].set_xlabel('Model')
axes[0].set_ylabel('RMSE')
axes[0].set_title('RMSE Comparison')
axes[0].set_xticks(x)
axes[0].set_xticklabels(results_df['Model'])
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Direction Accuracy karşılaştırması
axes[1].bar(x - width/2, results_df['Train_DirAcc'], width, label='Train', alpha=0.8)
axes[1].bar(x + width/2, results_df['Val_DirAcc'], width, label='Val', alpha=0.8)
axes[1].set_xlabel('Model')
axes[1].set_ylabel('Direction Accuracy')
axes[1].set_title('Direction Accuracy Comparison')
axes[1].set_xticks(x)
axes[1].set_xticklabels(results_df['Model'])
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Feature Importance (LightGBM)

In [ ]:
# LightGBM feature importance
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': lgb_model.feature_importance(importance_type='gain')
}).sort_values('importance', ascending=False)

# Top 20 features
top_features = importance_df.head(20)

plt.figure(figsize=(10, 8))
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Importance (Gain)')
plt.title('Top 20 Features - LightGBM')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop 10 Features:")
print(top_features.head(10).to_string(index=False))

## 7. SHAP Feature Importance

In [ ]:
# Compute SHAP values (with sampling)
sample_size = min(500, len(X_val_scaled))
sample_idx = np.random.choice(len(X_val_scaled), sample_size, replace=False)
X_sample = X_val_scaled[sample_idx]

explainer = shap.TreeExplainer(lgb_model)
shap_values = explainer.shap_values(X_sample)

print(f"SHAP values computed: {shap_values.shape}")

In [ ]:
# SHAP summary plot
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_sample, feature_names=X_train.columns, max_display=20, show=False)
plt.tight_layout()
plt.show()

## 8. Ensemble & Test Submission

In [ ]:
# Test predictions (retrain with full train data)
X_full_scaled = scaler.fit_transform(X_train)

# Ridge
ridge_full = Ridge(alpha=1.0, random_state=42)
ridge_full.fit(X_full_scaled, y_train)
ridge_test_pred = ridge_full.predict(X_test_scaled)

# LightGBM
lgb_full_train = lgb.Dataset(X_full_scaled, y_train)
lgb_full = lgb.train(lgb_params, lgb_full_train, num_boost_round=lgb_model.best_iteration)
lgb_test_pred = lgb_full.predict(X_test_scaled)

# XGBoost
xgb_full_train = xgb.DMatrix(X_full_scaled, label=y_train)
xgb_full = xgb.train(xgb_params, xgb_full_train, num_boost_round=xgb_model.best_iteration)
xgb_test = xgb.DMatrix(X_test_scaled)
xgb_test_pred = xgb_full.predict(xgb_test)

print("Test predictions completed")

In [ ]:
# Ensemble (Weighted Average) — weight by validation RMSE
weights = {
    'ridge': 1.0 / ridge_val_metrics['RMSE'],
    'lgb': 1.0 / lgb_val_metrics['RMSE'],
    'xgb': 1.0 / xgb_val_metrics['RMSE']
}

total_weight = sum(weights.values())
weights = {k: v/total_weight for k, v in weights.items()}

print("Ensemble Weights:")
for model, weight in weights.items():
    print(f"  {model}: {weight:.4f}")

# Ensemble prediction
ensemble_pred = (
    weights['ridge'] * ridge_test_pred +
    weights['lgb'] * lgb_test_pred +
    weights['xgb'] * xgb_test_pred
)

print(f"\nEnsemble prediction range: [{ensemble_pred.min():.4f}, {ensemble_pred.max():.4f}]")

In [ ]:
# Create submission file
submission_df = pd.DataFrame({
    'id': test_enhanced[date_col],
    'prediction': ensemble_pred
})

submission_df.to_csv('submission.csv', index=False)
print("✓ submission.csv created")
print(f"\nFirst 5 rows:")
submission_df.head(10)

## 9. Prediction Distribution

In [ ]:
# Test tahminlerinin dağılımı
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Ridge
axes[0, 0].hist(ridge_test_pred, bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title(f'Ridge Predictions (μ={ridge_test_pred.mean():.4f}, σ={ridge_test_pred.std():.4f})')
axes[0, 0].set_xlabel('Prediction')

# LightGBM
axes[0, 1].hist(lgb_test_pred, bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[0, 1].set_title(f'LightGBM Predictions (μ={lgb_test_pred.mean():.4f}, σ={lgb_test_pred.std():.4f})')
axes[0, 1].set_xlabel('Prediction')

# XGBoost
axes[1, 0].hist(xgb_test_pred, bins=50, edgecolor='black', alpha=0.7, color='green')
axes[1, 0].set_title(f'XGBoost Predictions (μ={xgb_test_pred.mean():.4f}, σ={xgb_test_pred.std():.4f})')
axes[1, 0].set_xlabel('Prediction')

# Ensemble
axes[1, 1].hist(ensemble_pred, bins=50, edgecolor='black', alpha=0.7, color='red')
axes[1, 1].set_title(f'Ensemble Predictions (μ={ensemble_pred.mean():.4f}, σ={ensemble_pred.std():.4f})')
axes[1, 1].set_xlabel('Prediction')

plt.tight_layout()
plt.show()

## 10. Summary

### Completed Steps:
1. ✓ Data loading and exploration
2. ✓ Feature engineering (RSI, MACD, EMA, Bollinger Bands)
3. ✓ Model training (Ridge, LightGBM, XGBoost)
4. ✓ Model comparison and metrics
5. ✓ Feature importance (gain + SHAP)
6. ✓ Ensemble modeling
7. ✓ Test submission creation

### Next Steps:
- Hyperparameter tuning (Optuna)
- Robust evaluation with cross-validation
- Stacking ensemble
- Backtest and risk metrics